#**Analysis/Parsing of the SSL Certificate**


#Certificate chain:


*   Starts with certiftcate for the *client/server*
*   Ends with *self-signed* certificate





#Certificate for the server

In [1]:
#install and import of required libraries
!pip install pyopenssl
import ssl,socket,certifi,os,json
from cryptography import x509
from cryptography.hazmat.backends import default_backend

In [2]:
#host and port of the url to analyse SSL certificate
host = 'pilot.wright.edu'
port = 443

#creating a tuple of address using host and port
address = (host,port)

#The function get_server_certificate() returns the digital certificate of a server host
encoded_certificate = ssl.get_server_certificate(address)

                                      #or

#Validates it if root certificates are provided
#encoded_certificate = ssl.get_server_certificate(address,ca_certs=os.path.relpath(certifi.where()))


In [3]:
#PEM formatted key, the base64 encoded x509 ASN.1 key           Output Type:String
print(encoded_certificate)

-----BEGIN CERTIFICATE-----
MIIFzDCCBLSgAwIBAgIQDRxkUnxA4N1u5TtsHIE1fTANBgkqhkiG9w0BAQsFADA8
MQswCQYDVQQGEwJVUzEPMA0GA1UEChMGQW1hem9uMRwwGgYDVQQDExNBbWF6b24g
UlNBIDIwNDggTTAxMB4XDTIzMDkyNjAwMDAwMFoXDTI0MTAyNDIzNTk1OVowGzEZ
MBcGA1UEAxMQcGlsb3Qud3JpZ2h0LmVkdTCCASIwDQYJKoZIhvcNAQEBBQADggEP
ADCCAQoCggEBAL5bWz7piF90iXhPx0AUzL3C7l9u3KCAzebKOq7bFUI0lX2J+xBM
Lfw4UWBXpePWzpgCZuOHCr+Pl29N8QfFap4H9gBFD+DEfRcecD6/l27YTt7YfMvC
4Qs0TXH/yLANB3wjNpq6vvGZ0O5N/ViVlVN34sa4H/I1z10AFKrG5c0L7kC3iAF0
+HWz3bTPlSxk6HSxUZ4TwUntNRfl0auwP9LfeG1TB8dp4DZZW0d9G39siY9pKd9d
fbsIqQQECKWq7LwDKSUw5EKSyDgh1CPyYychriYsj9x0QAbca3OYvAMdBuE4Vm83
VO7xTFHF+vDC1IL9sUPlL4WxeMWWpucqbFMCAwEAAaOCAukwggLlMB8GA1UdIwQY
MBaAFIG4DmOKiRIY5fo7O1CVn+blkBOFMB0GA1UdDgQWBBQs+Po0dDt2KqmSxaQb
1r119uH1YTAbBgNVHREEFDASghBwaWxvdC53cmlnaHQuZWR1MBMGA1UdIAQMMAow
CAYGZ4EMAQIBMA4GA1UdDwEB/wQEAwIFoDAdBgNVHSUEFjAUBggrBgEFBQcDAQYI
KwYBBQUHAwIwOwYDVR0fBDQwMjAwoC6gLIYqaHR0cDovL2NybC5yMm0wMS5hbWF6
b250cnVzdC5jb20vcjJtMDEuY3JsMHUGCCsGAQUFBwEBBGkwZzAtBggrBgEFBQ

#Decoding the certificate

In [4]:
#string type to bytes conversion
encoded_certificate = bytes(encoded_certificate, 'utf-8')

#decoding PEM using x509 of cryptography
decoded_certificate = x509.load_pem_x509_certificate(encoded_certificate)

In [5]:
#decoded cerificate
print(decoded_certificate)

<Certificate(subject=<Name(CN=pilot.wright.edu)>, ...)>


#Extracting the information from the decoded certificate

In [6]:
#basic information of the certificate
print("Not Valid After:            ",decoded_certificate.not_valid_after)
print("Not Valid Before:           ",decoded_certificate.not_valid_before)
print("Issuer:                     ",decoded_certificate.issuer)
print("Serial Number:              ",decoded_certificate.serial_number)
print("Subject:                    ",decoded_certificate.subject)
print("Version:                    ",decoded_certificate.version)

Not Valid After:             2024-10-24 23:59:59
Not Valid Before:            2023-09-26 00:00:00
Issuer:                      <Name(C=US,O=Amazon,CN=Amazon RSA 2048 M01)>
Serial Number:               17427383033367147223182009405347542397
Subject:                     <Name(CN=pilot.wright.edu)>
Version:                     Version.v3


In [7]:
#names of different properties
print('Display of different feature names of the certificate - (Keys):\n')
for extension in decoded_certificate.extensions:
  print("\t",str(extension.oid).split("=")[2][:-2])

Display of different feature names of the certificate - (Keys):

	 authorityKeyIdentifier
	 subjectKeyIdentifier
	 subjectAltName
	 certificatePolicies
	 keyUsage
	 extendedKeyUsage
	 cRLDistributionPoints
	 authorityInfoAccess
	 basicConstraints
	 signedCertificateTimestampList


In [8]:
#Subject Alternative Names
for name in decoded_certificate.extensions[5].value:
  print(str(name).split('=')[1][:-2])

1.3.6.1.5.5.7.3.1, na
1.3.6.1.5.5.7.3.2, na


#Certificate for the CA (Certificate Authority)

In [9]:
#creating SSL context
context = ssl.SSLContext(ssl.PROTOCOL_TLSv1_2)

#ensure to check hostname by the flag 'True' for socket connection
context.check_hostname = True

#loading all default certificates for the context
context.load_default_certs()

<ipython-input-9-677bffa69388>:2: DeprecationWarning: ssl.PROTOCOL_TLSv1_2 is deprecated
  context = ssl.SSLContext(ssl.PROTOCOL_TLSv1_2)


In [10]:
#creating socket
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

#wrapping socket and context
ssl_sock = context.wrap_socket(s, server_hostname=host)

#establishing connection to the address
ssl_sock.connect(address)

In [11]:
#the function ssl_sock.getpeercert() returns the disctionary instance of different keys/principals like issuer,subject
dictionary = ssl_sock.getpeercert()
print(dictionary)
#printing all the information of the certificate from the above dictionary
print("Issuer: ",dictionary['issuer'])
print("\nSubject: ",dictionary['subject'])
print("\nVersion: ",dictionary['version'])
print("\nSerial Number: ",dictionary['serialNumber'])
print("\nNot Valid After: ",dictionary['notAfter'])
print("\nNot Valid Before:  ",dictionary['notBefore'])
print("\nOCSP: ",dictionary['OCSP'])
print("\ncaIssuers: ",dictionary['caIssuers'])
print("\ncrlDistributionPoints: ",dictionary['crlDistributionPoints'])
print("\nsubjectAltName:  ",end="")
print("\t",dictionary['subjectAltName'][0][1])
for i in range(1,len(dictionary['subjectAltName'])):
  print("\t\t\t",dictionary['subjectAltName'][i][1])

{'subject': ((('commonName', 'pilot.wright.edu'),),), 'issuer': ((('countryName', 'US'),), (('organizationName', 'Amazon'),), (('commonName', 'Amazon RSA 2048 M01'),)), 'version': 3, 'serialNumber': '0D1C64527C40E0DD6EE53B6C1C81357D', 'notBefore': 'Sep 26 00:00:00 2023 GMT', 'notAfter': 'Oct 24 23:59:59 2024 GMT', 'subjectAltName': (('DNS', 'pilot.wright.edu'),), 'OCSP': ('http://ocsp.r2m01.amazontrust.com',), 'caIssuers': ('http://crt.r2m01.amazontrust.com/r2m01.cer',), 'crlDistributionPoints': ('http://crl.r2m01.amazontrust.com/r2m01.crl',)}
Issuer:  ((('countryName', 'US'),), (('organizationName', 'Amazon'),), (('commonName', 'Amazon RSA 2048 M01'),))

Subject:  ((('commonName', 'pilot.wright.edu'),),)

Version:  3

Serial Number:  0D1C64527C40E0DD6EE53B6C1C81357D

Not Valid After:  Oct 24 23:59:59 2024 GMT

Not Valid Before:   Sep 26 00:00:00 2023 GMT

OCSP:  ('http://ocsp.r2m01.amazontrust.com',)

caIssuers:  ('http://crt.r2m01.amazontrust.com/r2m01.cer',)

crlDistributionPoints: 

#Resources:

*Basix syntax for SSL*:
https://pythontic.com/ssl

*Decoding PEM using crypography*: https://cryptography.io/en/latest/x509/reference/#loading-certificates

*Library for entire SSL*: https://docs.python.org/3/library/ssl.html

Exclusivily for certificate chains: https://docs.python.org/3/library/ssl.html#certificates


